# **Live Model Server Testing**

Test model server via HTTP calls.<br>
To see how the model is trained or how the data-set is generated, check out `sklearn_classifier` function from the function marketplace repository

### **Steps**

1. [Setup function parameters](#Setup-function-parameters)
2. [Importing the function](#Importing-the-function)
3. [Running the function locally](#Running-the-function-locally)
4. [Running the function remotely](#Running-the-function-remotely)

In [1]:
import warnings
warnings.filterwarnings("ignore")

### **Setup function parameters**

In [3]:
# Declaring the model's path that we are going to test
models_path = 'https://s3.wasabisys.com/iguazio/models/function-marketplace-models/test_classifier/RandomForestClassifier.pkl'
# Testing with dataset
data_path = 'https://s3.wasabisys.com/iguazio/data/function-marketplace-data/sklearn_classifier/iris_dataset.csv'

In [4]:
import mlrun
mlrun.set_environment(project='function-marketplace')

# deploying v2_model_server in order to get an http endpoint to test
# importing the function from the hub
fn = mlrun.import_function("hub://v2_model_server")
fn.apply(mlrun.auto_mount())

# Adding the model 
fn.add_model(key='RandomForestClassifier', model_path=models_path ,class_name='ClassifierModel')
address = fn.deploy()

> 2021-10-18 05:03:08,020 [info] loaded project function-marketplace from MLRun DB
> 2021-10-18 05:03:08,791 [info] Starting remote function deploy
2021-10-18 05:03:08  (info) Deploying function
2021-10-18 05:03:08  (info) Building
2021-10-18 05:03:08  (info) Staging files and preparing base images
2021-10-18 05:03:08  (info) Building processor image
2021-10-18 05:03:10  (info) Build complete
2021-10-18 05:03:18  (info) Function deploy complete
> 2021-10-18 05:03:18,932 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-function-marketplace-v2-model-server.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['default-tenant.app.dev39.lab.iguazeng.com:31003']}


### **Importing the function**

In [10]:
# Importing the function from the hub
fn = mlrun.import_function("hub://v2_model_tester")
fn.apply(mlrun.auto_mount())

### **Running the function locally**

In [12]:
tester_run = fn.run(name='model_server_tester',
                    handler = 'model_server_tester', # default value is 'handler', therefore specifying function entry point.
                    params={'addr': address, 'model': 'RandomForestClassifier'},
                    inputs={'table': data_path},
                    local=True)

> 2021-10-18 05:10:09,850 [info] starting run model_server_tester uid=c5773ef3ebc544d390c3b66cce7d7665 DB=http://mlrun-api:8080
> 2021-10-18 05:10:11,386 [info] testing with dataset against http://default-tenant.app.dev39.lab.iguazeng.com:31003, model: RandomForestClassifier
{'id': '674dc477-0281-4aaa-b98c-e74998ac295d', 'model_name': 'RandomForestClassifier', 'outputs': [0]}
{'id': '8ff10778-834b-4920-a455-2d638d125116', 'model_name': 'RandomForestClassifier', 'outputs': [1]}
{'id': 'c4a03a43-2715-4772-b88b-6db8a222f8d4', 'model_name': 'RandomForestClassifier', 'outputs': [1]}
{'id': '03d3b15c-843d-4ba3-af90-ec5aaf5ccec4', 'model_name': 'RandomForestClassifier', 'outputs': [2]}
{'id': '10caf431-7375-4d0a-840c-6d4bd479d2e3', 'model_name': 'RandomForestClassifier', 'outputs': [0]}
{'id': '7b9c8753-6ff7-46f4-a77c-e91a2f49826d', 'model_name': 'RandomForestClassifier', 'outputs': [2]}
{'id': 'e08f322b-f732-4d88-9911-a6b4d55b6b51', 'model_name': 'RandomForestClassifier', 'outputs': [0]}
{'i

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
function-marketplace,...ce7d7665,0,Oct 18 05:10:10,completed,model_server_tester,v3io_user=danikind=owner=danihost=jupyter-dani-6bfbd76d96-zxx6f,table,addr=http://default-tenant.app.dev39.lab.iguazeng.com:31003model=RandomForestClassifier,total_tests=20errors=0match=9avg_latency=26833min_latency=19420max_latency=152063,latency


> 2021-10-18 05:10:12,045 [info] run executed, status=completed


In [20]:
tester_run.artifact('latency').show()

### **Running the function remotely**

In [21]:
tester_run = fn.run(name='model_server_tester',
                    handler = 'model_server_tester', # default value is 'handler', therefore specifying function entry point.
                    params={'addr': address, 'model': 'RandomForestClassifier'},
                    inputs={'table': data_path},
                    local=False)

> 2021-10-18 05:12:30,250 [info] starting run model_server_tester uid=a3ffac1155564332a828961fa66ab30c DB=http://mlrun-api:8080
> 2021-10-18 05:12:30,380 [info] Job is running in the background, pod: model-server-tester-thhll
> 2021-10-18 05:12:36,763 [info] testing with dataset against http://default-tenant.app.dev39.lab.iguazeng.com:31003, model: RandomForestClassifier
{'id': '4318bfe9-ec16-43ad-9990-d637a87275a7', 'model_name': 'RandomForestClassifier', 'outputs': [2]}
{'id': '71ba6a72-dd4c-4cb0-a877-39dc8d739377', 'model_name': 'RandomForestClassifier', 'outputs': [2]}
{'id': 'c5359476-b0b1-4dd8-8ce7-562addb02929', 'model_name': 'RandomForestClassifier', 'outputs': [0]}
{'id': '0679c92f-9f92-46bc-a773-8df73de2c5de', 'model_name': 'RandomForestClassifier', 'outputs': [2]}
{'id': '2c272335-ee43-458e-b662-c236d29f62dc', 'model_name': 'RandomForestClassifier', 'outputs': [2]}
{'id': 'c9e600b5-36ef-4366-b771-519a3037bb2e', 'model_name': 'RandomForestClassifier', 'outputs': [1]}
{'id': '

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
function-marketplace,...a66ab30c,0,Oct 18 05:12:35,completed,model_server_tester,v3io_user=danikind=jobowner=danihost=model-server-tester-thhll,table,addr=http://default-tenant.app.dev39.lab.iguazeng.com:31003model=RandomForestClassifier,total_tests=20errors=0match=6avg_latency=29752min_latency=27708max_latency=33304,latency


> 2021-10-18 05:12:39,586 [info] run executed, status=completed


[Back to the top](#Live-Model-Server-Testing)